In [0]:
import numpy as np
import pandas as pd
import plotly as px
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from datetime import datetime
import time

import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.functions import isnan, when, count, col

from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, StandardScaler, OneHotEncoder, SQLTransformer
from pyspark.ml.classification import LogisticRegression 
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.sql import Window

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import mlflow
import mlflow.spark

Get count of incoming delayed flight so far each day to each airport.  Also average incoming delay so far.

In [0]:
data = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/train_final")

print(data.count())
display(data)

16659291

YEAR QUARTER MONTH DAY_OF_MONTH DAY_OF_WEEK FL_DATE OP_UNIQUE_CARRIER OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM ORIGIN_AIRPORT_ID ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_MARKET_ID ORIGIN ORIGIN_CITY_NAME ORIGIN_STATE_ABR ORIGIN_STATE_FIPS ORIGIN_STATE_NM ORIGIN_WAC DEST_AIRPORT_ID DEST_AIRPORT_SEQ_ID DEST_CITY_MARKET_ID DEST DEST_CITY_NAME DEST_STATE_ABR DEST_STATE_FIPS DEST_STATE_NM DEST_WAC CRS_DEP_TIME DEP_TIME DEP_DELAY DEP_DELAY_NEW DEP_DEL15 DEP_DELAY_GROUP DEP_TIME_BLK TAXI_OUT WHEELS_OFF WHEELS_ON TAXI_IN CRS_ARR_TIME ARR_TIME ARR_DELAY ARR_DELAY_NEW ARR_DEL15 ARR_DELAY_GROUP ARR_TIME_BLK CANCELLED DIVERTED CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME AIR_TIME FLIGHTS DISTANCE DISTANCE_GROUP DIV_AIRPORT_LANDINGS ORIGIN_TZ DEST_TZ DEP_MIN DEP_HOUR ARR_MIN ARR_HOUR ORIGIN_TS ORIGIN_UTC DEST_TS DEST_UTC ORIGIN_FLIGHT_COUNT DEST_FLIGHT_COUNT ORIGIN_STATION ORIGIN_STATION_NAME DEST_STATION DEST_STATION_NAME PAGERANK ORIGIN_UTC_ADJ_MIN ORIGIN_UTC_ADJ_MAX ORIGIN_MAX_DATE DEST_MAX_DATE DELAYS_SO_FAR CRS_ELAPSED_TIME_AVG_DIFF WEST_TO_EAST AVG_WND_SPEED_ORIGIN AVG_CIG_HEIGHT_ORIGIN MIN_CIG_HEIGHT_ORIGIN AVG_VIS_DIS_ORIGIN MIN_VIS_DIS_ORIGIN AVG_TMP_DEG_ORIGIN AVG_DEW_DEG_ORIGIN AVG_SLP_ORIGIN AVG_WND_SPEED_DEST AVG_CIG_HEIGHT_DEST MIN_CIG_HEIGHT_DEST AVG_VIS_DIS_DEST MIN_VIS_DIS_DEST AVG_TMP_DEG_DEST AVG_DEW_DEG_DEST AVG_SLP_DEST MINUTES_AFTER_MIDNIGHT_ORIGIN MINUTES_AFTER_MIDNIGHT_DEST 2015 2 5 14 4 2015-05-14 EV 20366 EV N11547 6144 10135 1013503 30135 ABE Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 1727 1746 19.0 19.0 1.0 1 1700-1759 10.0 1756 1850 7.0 1836 1857 21.0 21.0 1.0 1 1800-1859 0.0 0.0 129.0 131.0 114.0 1.0 654.0 3 0 America/New_York America/Chicago 27 17 36 18 2015-05-14T17:27:00.000+0000 2015-05-14T21:27:00.000+0000 2015-05-14T18:36:00.000+0000 2015-05-14T23:36:00.000+0000 18 1847 72517014737 LEHIGH VALLEY INTERNATIONAL A 72530094846 CHICAGO O'HARE INTERNATIONAL 9.123668620151711E-4 2015-05-12T19:27:00.000+0000 2015-05-14T19:27:00.000+0000 2015-05-14T18:51:00.000+0000 2015-05-14T18:51:00.000+0000 1.0 -8.253675213675223 0 24.970625000000002 22000.0 22000.0 16081.076923076924 16000.0 190.84615384615384 2.625 10283.76 41.7962962962963 19508.624913355172 2286.0 16089.836734693878 16000.0 228.5703125 103.38513513513513 10235.306918822756 1287 1416 2016 1 1 19 2 2016-01-19 EV 20366 EV N11565 6021 10135 1013503 30135 ABE Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 13930 1393004 30977 ORD Chicago, IL IL 17 Illinois 41 624 620 -4.0 0.0 0.0 -1 0600-0659 18.0 638 729 5.0 744 734 -10.0 0.0 0.0 -1 0700-0759 0.0 0.0 140.0 134.0 111.0 1.0 655.0 3 0 America/New_York America/Chicago 24 6 44 7 2016-01-19T06:24:00.000+0000 2016-01-19T11:24:00.000+0000 2016-01-19T07:44:00.000+0000 2016-01-19T13:44:00.000+0000 11 1237 72517014737 LEHIGH VALLEY INTERNATIONAL A 72530094846 CHICAGO O'HARE INTERNATIONAL 9.123668620151711E-4 2016-01-17T09:24:00.000+0000 2016-01-19T09:24:00.000+0000 2016-01-19T08:51:00.000+0000 2016-01-19T08:51:00.000+0000 0.0 2.7463247863247773 0 57.62162162162162 18698.71875 823.0 15998.35294117647 14484.0 -104.25 -168.83333333333334 10197.02380952381 24.391304347826086 18773.735099337748 518.0 15854.662420382165 2816.0 -94.39097744360902 -160.4240506329114 10307.8188743289 684 824 2015 4 11 10 2 2015-11-10 EV 20366 EV N12567 6144 10135 1013503 30135 ABE Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 13930 1393004 30977 ORD Chicago, IL IL 17 Illinois 41 1713 1716 3.0 3.0 0.0 0 1700-1759 8.0 1724 1806 9.0 1829 1815 -14.0 0.0 0.0 -1 1800-1859 0.0 0.0 136.0 119.0 102.0 1.0 655.0 3 0 America/New_York America/Chicago 13 17 29 18 2015-11-10T17:13:00.000+0000 2015-11-10T22:13:00.000+0000 2015-11-10T18:29:00.000+0000 2015-11-11T00:29:00.000+0000 18 1828 72517014737 LEHIGH VALLEY INTERNATIONAL A 72530094846 CHICAGO O'HARE INTERNATIONAL 9.123668620151711E-4 2015-11-08T20:13:00.000+0000 2015-11-10T20:13:00.000+0000 2015-11-10T19:51:00.000+0000 2015-11-

try on sample subset

In [0]:
test_sample = data.orderBy('ORIGIN_UTC').limit(10000).withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))

In [0]:
test_sample.registerTempTable('sample_table')

In [0]:
count_incoming_delays = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(sum(d2.ARR_DEL15),0) as incoming_delays
      FROM sample_table as d1
      LEFT JOIN sample_table d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

display(count_incoming_delays)

ORIGIN ORIGIN_UTC incoming_delays PBG 2015-01-01T06:55:00.000+0000 0.0 IAG 2015-01-01T07:00:00.000+0000 0.0 MSP 2015-01-01T07:15:00.000+0000 0.0 DEN 2015-01-01T07:30:00.000+0000 0.0 SLC 2015-01-01T07:40:00.000+0000 0.0 LAX 2015-01-01T08:10:00.000+0000 0.0 SFO 2015-01-01T08:20:00.000+0000 0.0 DEN 2015-01-01T08:20:00.000+0000 0.0 LAX 2015-01-01T08:20:00.000+0000 0.0 SFO 2015-01-01T08:25:00.000+0000 0.0 PHX 2015-01-01T08:25:00.000+0000 0.0 SEA 2015-01-01T08:25:00.000+0000 0.0 LAS 2015-01-01T08:25:00.000+0000 0.0 PHX 2015-01-01T08:27:00.000+0000 0.0 SFO 2015-01-01T08:30:00.000+0000 0.0 LAS 2015-01-01T08:30:00.000+0000 0.0 LAX 2015-01-01T08:30:00.000+0000 0.0 LAX 2015-01-01T08:35:00.000+0000 0.0 LAS 2015-01-01T08:35:00.000+0000 0.0 SEA 2015-01-01T08:40:00.000+0000 0.0 SLC 2015-01-01T08:40:00.000+0000 0.0 SFO 2015-01-01T08:48:00.000+0000 0.0 PDX 2015-01-01T08:50:00.000+0000 0.0 LAS 2015-01-01T08:55:00.000+0000 0.0 PHX 2015-01-01T08:59:00.000+0000 0.0 SEA 2015-01-01T09:00:00.000+0000 0.0 LAS 2015-01-01T09:03:00.000+0000 0.0 LAS 2015-01-01T09:05:00.000+0000 0.0 ANC 2015-01-01T09:05:00.000+0000 0.0 LAX 2015-01-01T09:05:00.000+0000 0.0 LAX 2015-01-01T09:15:00.000+0000 0.0 PHX 2015-01-01T09:20:00.000+0000 0.0 LAS 2015-01-01T09:20:00.000+0000 0.0 LAS 2015-01-01T09:44:00.000+0000 0.0 LAS 2015-01-01T09:45:00.000+0000 0.0 ANC 2015-01-01T09:45:00.000+0000 0.0 ANC 2015-01-01T09:50:00.000+0000 0.0 MCO 2015-01-01T10:10:00.000+0000 0.0 BOS 2015-01-01T10:10:00.000+0000 0.0 FAI 2015-01-01T10:15:00.000+0000 0.0 BOS 2015-01-01T10:15:00.000+0000 0.0 SBN 2015-01-01T10:30:00.000+0000 0.0 PHL 2015-01-01T10:30:00.000+0000 0.0 PWM 2015-01-01T10:35:00.000+0000 0.0 BUF 2015-01-01T10:35:00.000+0000 0.0 IAD 2015-01-01T10:35:00.000+0000 0.0 JAX 2015-01-01T10:40:00.000+0000 0.0 ATL 2015-01-01T10:40:00.000+0000 0.0 BOS 2015-01-01T10:40:00.000+0000 0.0 JFK 2015-01-01T10:45:00.000+0000 0.0 DCA 2015-01-01T10:45:00.000+0000 0.0 BOS 2015-01-01T10:45:00.000+0000 0.0 CHS 2015-01-01T10:45:00.000+0000 0.0 BOS 2015-01-01T10:49:00.000+0000 0.0 RIC 2015-01-01T10:50:00.000+0000 0.0 IND 2015-01-01T10:50:00.000+0000 0.0 BDL 2015-01-01T10:50:00.000+0000 0.0 MCO 2015-01-01T10:50:00.000+0000 0.0 CLE 2015-01-01T10:50:00.000+0000 0.0 BOS 2015-01-01T10:50:00.000+0000 0.0 GSP 2015-01-01T10:50:00.000+0000 0.0 ANC 2015-01-01T10:52:00.000+0000 0.0 FLL 2015-01-01T10:53:00.000+0000 0.0 TPA 2015-01-01T10:55:00.000+0000 0.0 MCO 2015-01-01T10:55:00.000+0000 0.0 BUF 2015-01-01T10:55:00.000+0000 0.0 SYR 2015-01-01T10:55:00.000+0000 0.0 ANC 2015-01-01T10:55:00.000+0000 0.0 LAN 2015-01-01T10:55:00.000+0000 0.0 BOS 2015-01-01T10:55:00.000+0000 0.0 ROC 2015-01-01T10:55:00.000+0000 0.0 GSO 2015-01-01T10:56:00.000+0000 0.0 EWR 2015-01-01T10:59:00.000+0000 0.0 CVG 2015-01-01T11:00:00.000+0000 0.0 LGA 2015-01-01T11:00:00.000+0000 0.0 MIA 2015-01-01T11:00:00.000+0000 0.0 JFK 2015-01-01T11:00:00.000+0000 0.0 EWR 2015-01-01T11:00:00.000+0000 0.0 TPA 2015-01-01T11:00:00.000+0000 0.0 PIT 2015-01-01T11:00:00.000+0000 0.0 ACY 2015-01-01T11:00:00.000+0000 0.0 MCO 2015-01-01T11:00:00.000+0000 0.0 DCA 2015-01-01T11:00:00.000+0000 0.0 ORD 2015-01-01T11:00:00.000+0000 0.0 BOS 2015-01-01T11:00:00.000+0000 0.0 BWI 2015-01-01T11:00:00.000+0000 0.0 FLL 2015-01-01T11:00:00.000+0000 0.0 MHT 2015-01-01T11:00:00.000+0000 0.0 TTN 2015-01-01T11:00:00.000+0000 0.0 PBI 2015-01-01T11:00:00.000+0000 0.0 DTW 2015-01-01T11:00:00.000+0000 0.0 ABE 2015-01-01T11:00:00.000+0000 0.0 RSW 2015-01-01T11:00:00.000+0000 0.0 CLT 2015-01-01T11:00:00.000+0000 0.0 ESC 2015-01-01T11:00:00.000+0000 0.0 CMH 2015-01-01T11:00:00.000+0000 0.0 ATL 2015-01-01T11:00:00.000+0000 0.0 TYS 2015-01-01T11:00:00.000+0000 0.0 CLE 2015-01-01T11:00:00.000+0000 0.0 JAX 2015-01-01T11:00:00.000+0000 0.0 SAV 2015-01-01T11:00:00.000+0000 0.0 RDU 2015-01-01T11:00:00.000+0000 0.0 ERI 2015-01-01T11:00:00.000+0000 0.0 PVD 2015-01-01T11:00:00.000+0000 0.0 ANC 2015-01-01T11:00:00.000+0000 0.0 RIC 2015-01-01T11:00:00.000+0000 0.0 LEX 2015-01-01T11:00:00.000+0000 0.0 OAJ 201

In [0]:
id_group_pd = count_incoming_delays.groupBy("incoming_delays").count().toPandas()
id_group_pd.head(20)

Out[17]:

incoming_delays 
 count 
 
 
 
 
 0 
 0.0 
 3451 
 
 
 1 
 1.0 
 564 
 
 
 2 
 2.0 
 424 
 
 
 3 
 3.0 
 251 
 
 
 4 
 4.0 
 273 
 
 
 5 
 8.0 
 167 
 
 
 6 
 6.0 
 145 
 
 
 7 
 9.0 
 73 
 
 
 8 
 12.0 
 65 
 
 
 9 
 5.0 
 129 
 
 
 10 
 10.0 
 107 
 
 
 11 
 30.0 
 37 
 
 
 12 
 20.0 
 49 
 
 
 13 
 15.0 
 52 
 
 
 14 
 25.0 
 9 
 
 
 15 
 7.0 
 73 
 
 
 16 
 64.0 
 4 
 
 
 17 
 16.0 
 42 
 
 
 18 
 14.0 
 33 
 
 
 19 
 40.0 
 17

In [0]:
avg_incoming_delay = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(avg(d2.ARR_DELAY),0) as avg_incoming_delay
      FROM sample_table as d1
      LEFT JOIN sample_table d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

display(avg_incoming_delay)

ORIGIN ORIGIN_UTC avg_incoming_delay PBG 2015-01-01T06:55:00.000+0000 0.0 IAG 2015-01-01T07:00:00.000+0000 0.0 MSP 2015-01-01T07:15:00.000+0000 0.0 DEN 2015-01-01T07:30:00.000+0000 0.0 SLC 2015-01-01T07:40:00.000+0000 0.0 LAX 2015-01-01T08:10:00.000+0000 0.0 SFO 2015-01-01T08:20:00.000+0000 0.0 LAX 2015-01-01T08:20:00.000+0000 0.0 DEN 2015-01-01T08:20:00.000+0000 0.0 SFO 2015-01-01T08:25:00.000+0000 0.0 SEA 2015-01-01T08:25:00.000+0000 0.0 PHX 2015-01-01T08:25:00.000+0000 0.0 LAS 2015-01-01T08:25:00.000+0000 0.0 PHX 2015-01-01T08:27:00.000+0000 0.0 SFO 2015-01-01T08:30:00.000+0000 0.0 LAS 2015-01-01T08:30:00.000+0000 0.0 LAX 2015-01-01T08:30:00.000+0000 0.0 LAX 2015-01-01T08:35:00.000+0000 0.0 LAS 2015-01-01T08:35:00.000+0000 0.0 SEA 2015-01-01T08:40:00.000+0000 0.0 SLC 2015-01-01T08:40:00.000+0000 0.0 SFO 2015-01-01T08:48:00.000+0000 0.0 PDX 2015-01-01T08:50:00.000+0000 0.0 LAS 2015-01-01T08:55:00.000+0000 0.0 PHX 2015-01-01T08:59:00.000+0000 0.0 SEA 2015-01-01T09:00:00.000+0000 0.0 LAS 2015-01-01T09:03:00.000+0000 0.0 LAX 2015-01-01T09:05:00.000+0000 0.0 ANC 2015-01-01T09:05:00.000+0000 0.0 LAS 2015-01-01T09:05:00.000+0000 0.0 LAX 2015-01-01T09:15:00.000+0000 0.0 PHX 2015-01-01T09:20:00.000+0000 0.0 LAS 2015-01-01T09:20:00.000+0000 0.0 LAS 2015-01-01T09:44:00.000+0000 0.0 ANC 2015-01-01T09:45:00.000+0000 0.0 LAS 2015-01-01T09:45:00.000+0000 0.0 ANC 2015-01-01T09:50:00.000+0000 0.0 MCO 2015-01-01T10:10:00.000+0000 0.0 BOS 2015-01-01T10:10:00.000+0000 0.0 FAI 2015-01-01T10:15:00.000+0000 0.0 BOS 2015-01-01T10:15:00.000+0000 0.0 SBN 2015-01-01T10:30:00.000+0000 0.0 PHL 2015-01-01T10:30:00.000+0000 0.0 PWM 2015-01-01T10:35:00.000+0000 0.0 IAD 2015-01-01T10:35:00.000+0000 0.0 BUF 2015-01-01T10:35:00.000+0000 0.0 ATL 2015-01-01T10:40:00.000+0000 0.0 BOS 2015-01-01T10:40:00.000+0000 0.0 JAX 2015-01-01T10:40:00.000+0000 0.0 BOS 2015-01-01T10:45:00.000+0000 0.0 CHS 2015-01-01T10:45:00.000+0000 0.0 JFK 2015-01-01T10:45:00.000+0000 0.0 DCA 2015-01-01T10:45:00.000+0000 0.0 BOS 2015-01-01T10:49:00.000+0000 0.0 IND 2015-01-01T10:50:00.000+0000 0.0 CLE 2015-01-01T10:50:00.000+0000 0.0 RIC 2015-01-01T10:50:00.000+0000 0.0 MCO 2015-01-01T10:50:00.000+0000 0.0 BDL 2015-01-01T10:50:00.000+0000 0.0 GSP 2015-01-01T10:50:00.000+0000 0.0 BOS 2015-01-01T10:50:00.000+0000 0.0 ANC 2015-01-01T10:52:00.000+0000 0.0 FLL 2015-01-01T10:53:00.000+0000 0.0 BUF 2015-01-01T10:55:00.000+0000 0.0 ROC 2015-01-01T10:55:00.000+0000 0.0 LAN 2015-01-01T10:55:00.000+0000 0.0 MCO 2015-01-01T10:55:00.000+0000 0.0 ANC 2015-01-01T10:55:00.000+0000 0.0 SYR 2015-01-01T10:55:00.000+0000 0.0 TPA 2015-01-01T10:55:00.000+0000 0.0 BOS 2015-01-01T10:55:00.000+0000 0.0 GSO 2015-01-01T10:56:00.000+0000 0.0 EWR 2015-01-01T10:59:00.000+0000 0.0 ERI 2015-01-01T11:00:00.000+0000 0.0 MCO 2015-01-01T11:00:00.000+0000 0.0 BOS 2015-01-01T11:00:00.000+0000 0.0 BWI 2015-01-01T11:00:00.000+0000 0.0 DCA 2015-01-01T11:00:00.000+0000 0.0 MIA 2015-01-01T11:00:00.000+0000 0.0 CLE 2015-01-01T11:00:00.000+0000 0.0 JAX 2015-01-01T11:00:00.000+0000 0.0 SAV 2015-01-01T11:00:00.000+0000 0.0 EWR 2015-01-01T11:00:00.000+0000 0.0 TPA 2015-01-01T11:00:00.000+0000 0.0 PIT 2015-01-01T11:00:00.000+0000 0.0 ABE 2015-01-01T11:00:00.000+0000 0.0 FLL 2015-01-01T11:00:00.000+0000 0.0 TYS 2015-01-01T11:00:00.000+0000 0.0 LGA 2015-01-01T11:00:00.000+0000 0.0 PVD 2015-01-01T11:00:00.000+0000 0.0 DTW 2015-01-01T11:00:00.000+0000 0.0 MHT 2015-01-01T11:00:00.000+0000 0.0 TTN 2015-01-01T11:00:00.000+0000 0.0 CMH 2015-01-01T11:00:00.000+0000 0.0 ATL 2015-01-01T11:00:00.000+0000 0.0 JFK 2015-01-01T11:00:00.000+0000 0.0 RSW 2015-01-01T11:00:00.000+0000 0.0 CVG 2015-01-01T11:00:00.000+0000 0.0 ACY 2015-01-01T11:00:00.000+0000 0.0 LEX 2015-01-01T11:00:00.000+0000 0.0 OAJ 2015-01-01T11:00:00.000+0000 0.0 CLT 2015-01-01T11:00:00.000+0000 0.0 ORD 2015-01-01T11:00:00.000+0000 0.0 RDU 2015-01-01T11:00:00.000+0000 0.0 ANC 2015-01-01T11:00:00.000+0000 0.0 PBI 2015-01-01T11:00:00.000+0000 0.0 ESC 2015-01-01T11:00:00.000+0000 0.0 RIC 

In [0]:
test_sample_with_count = test_sample.join(count_incoming_delays, ['ORIGIN','ORIGIN_UTC'], 'left')
test_sample_with_count.count()

Out[18]: 10000

In [0]:
display(test_sample_with_count)

ORIGIN ORIGIN_UTC YEAR QUARTER MONTH DAY_OF_MONTH DAY_OF_WEEK FL_DATE OP_UNIQUE_CARRIER OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM ORIGIN_AIRPORT_ID ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_MARKET_ID ORIGIN_CITY_NAME ORIGIN_STATE_ABR ORIGIN_STATE_FIPS ORIGIN_STATE_NM ORIGIN_WAC DEST_AIRPORT_ID DEST_AIRPORT_SEQ_ID DEST_CITY_MARKET_ID DEST DEST_CITY_NAME DEST_STATE_ABR DEST_STATE_FIPS DEST_STATE_NM DEST_WAC CRS_DEP_TIME DEP_TIME DEP_DELAY DEP_DELAY_NEW DEP_DEL15 DEP_DELAY_GROUP DEP_TIME_BLK TAXI_OUT WHEELS_OFF WHEELS_ON TAXI_IN CRS_ARR_TIME ARR_TIME ARR_DELAY ARR_DELAY_NEW ARR_DEL15 ARR_DELAY_GROUP ARR_TIME_BLK CANCELLED DIVERTED CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME AIR_TIME FLIGHTS DISTANCE DISTANCE_GROUP DIV_AIRPORT_LANDINGS ORIGIN_TZ DEST_TZ DEP_MIN DEP_HOUR ARR_MIN ARR_HOUR ORIGIN_TS DEST_TS DEST_UTC ORIGIN_FLIGHT_COUNT DEST_FLIGHT_COUNT ORIGIN_STATION ORIGIN_STATION_NAME DEST_STATION DEST_STATION_NAME PAGERANK ORIGIN_UTC_ADJ_MIN ORIGIN_UTC_ADJ_MAX ORIGIN_MAX_DATE DEST_MAX_DATE DELAYS_SO_FAR CRS_ELAPSED_TIME_AVG_DIFF WEST_TO_EAST AVG_WND_SPEED_ORIGIN AVG_CIG_HEIGHT_ORIGIN MIN_CIG_HEIGHT_ORIGIN AVG_VIS_DIS_ORIGIN MIN_VIS_DIS_ORIGIN AVG_TMP_DEG_ORIGIN AVG_DEW_DEG_ORIGIN AVG_SLP_ORIGIN AVG_WND_SPEED_DEST AVG_CIG_HEIGHT_DEST MIN_CIG_HEIGHT_DEST AVG_VIS_DIS_DEST MIN_VIS_DIS_DEST AVG_TMP_DEG_DEST AVG_DEW_DEG_DEST AVG_SLP_DEST MINUTES_AFTER_MIDNIGHT_ORIGIN MINUTES_AFTER_MIDNIGHT_DEST DAY_ZERO incoming_delays ABE 2015-01-01T11:00:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N846AS 5103 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 11433 1143302 31295 DTW Detroit, MI MI 26 Michigan 43 600 552 -8.0 0.0 0.0 -1 0600-0659 12.0 604 728 5.0 753 733 -20.0 0.0 0.0 -2 0700-0759 0.0 0.0 113.0 101.0 84.0 1.0 425.0 2 0 America/New_York America/Detroit 0 6 53 7 2015-01-01T06:00:00.000+0000 2015-01-01T07:53:00.000+0000 2015-01-01T12:53:00.000+0000 5 340 72517014737 LEHIGH VALLEY INTERNATIONAL A 72537094847 DETROIT METRO WAYNE COUNTY AI 9.123668620151711E-4 2014-12-30T09:00:00.000+0000 2015-01-01T09:00:00.000+0000 2015-01-01T08:51:00.000+0000 2015-01-01T09:00:00.000+0000 0.0 4.296207965657047 0 29.305555555555557 20813.470588235294 1524.0 16093.0 16093.0 -40.04255319148936 -118.08571428571429 10213.7 53.51190476190476 14911.850683405892 640.0 16090.811764705883 16000.0 -69.98837209302326 -122.32558139534883 10192.760952380953 660 773 2015-01-01T00:00:00.000+0000 0.0 ABE 2015-01-01T11:35:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N136EV 5579 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 635 628 -7.0 0.0 0.0 -1 0600-0659 13.0 641 831 3.0 854 834 -20.0 0.0 0.0 -2 0800-0859 0.0 0.0 139.0 126.0 110.0 1.0 692.0 3 0 America/New_York America/New_York 35 6 54 8 2015-01-01T06:35:00.000+0000 2015-01-01T08:54:00.000+0000 2015-01-01T13:54:00.000+0000 5 1086 72517014737 LEHIGH VALLEY INTERNATIONAL A 72219013874 HARTSFIELD-JACKSON ATLANTA IN 9.123668620151711E-4 2014-12-30T09:35:00.000+0000 2015-01-01T09:35:00.000+0000 2015-01-01T08:51:00.000+0000 2015-01-01T08:52:00.000+0000 0.0 7.107549557148872 0 29.305555555555557 20813.470588235294 1524.0 16093.0 16093.0 -40.04255319148936 -118.08571428571429 10213.7 11.98913043478261 21082.112359550563 792.0 13132.08988764045 402.0 -10.024 -37.21348314606742 10275.547619047618 695 834 2015-01-01T00:00:00.000+0000 0.0 ABE 2015-01-01T21:00:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N131EV 5267 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 1600 1600 0.0 0.0 0.0 0 1600-1659 9.0 1609 1752 7.0 1820 1759 -21.0 0.0 0.0 -2 1800-1859 0.0 0.0 140.0 119.0 103.0 1.0 692.0 3 0 America/New_York America/New_York 0 16 20 18 2015-01-01T16:00:00.000+0000 2015-01-01T18:20:00.000+0000 2015-01-01T23:20:00.000+0000 5 1086 72517014737 LEHIGH VALLEY INTERNATIONAL A 72219013874 HARTSFIELD-JACKSON ATLANTA IN 9.123668620151711E-4 2014-12-30T19:0

In [0]:
test_sample_with_count_avg = test_sample_with_count.join(avg_incoming_delay, ['ORIGIN','ORIGIN_UTC'], 'left')
test_sample_with_count_avg.count()

Out[22]: 10000

In [0]:
display(test_sample_with_count_avg.drop('DAY_ZERO'))

ORIGIN ORIGIN_UTC YEAR QUARTER MONTH DAY_OF_MONTH DAY_OF_WEEK FL_DATE OP_UNIQUE_CARRIER OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM ORIGIN_AIRPORT_ID ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_MARKET_ID ORIGIN_CITY_NAME ORIGIN_STATE_ABR ORIGIN_STATE_FIPS ORIGIN_STATE_NM ORIGIN_WAC DEST_AIRPORT_ID DEST_AIRPORT_SEQ_ID DEST_CITY_MARKET_ID DEST DEST_CITY_NAME DEST_STATE_ABR DEST_STATE_FIPS DEST_STATE_NM DEST_WAC CRS_DEP_TIME DEP_TIME DEP_DELAY DEP_DELAY_NEW DEP_DEL15 DEP_DELAY_GROUP DEP_TIME_BLK TAXI_OUT WHEELS_OFF WHEELS_ON TAXI_IN CRS_ARR_TIME ARR_TIME ARR_DELAY ARR_DELAY_NEW ARR_DEL15 ARR_DELAY_GROUP ARR_TIME_BLK CANCELLED DIVERTED CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME AIR_TIME FLIGHTS DISTANCE DISTANCE_GROUP DIV_AIRPORT_LANDINGS ORIGIN_TZ DEST_TZ DEP_MIN DEP_HOUR ARR_MIN ARR_HOUR ORIGIN_TS DEST_TS DEST_UTC ORIGIN_FLIGHT_COUNT DEST_FLIGHT_COUNT ORIGIN_STATION ORIGIN_STATION_NAME DEST_STATION DEST_STATION_NAME PAGERANK ORIGIN_UTC_ADJ_MIN ORIGIN_UTC_ADJ_MAX ORIGIN_MAX_DATE DEST_MAX_DATE DELAYS_SO_FAR CRS_ELAPSED_TIME_AVG_DIFF WEST_TO_EAST AVG_WND_SPEED_ORIGIN AVG_CIG_HEIGHT_ORIGIN MIN_CIG_HEIGHT_ORIGIN AVG_VIS_DIS_ORIGIN MIN_VIS_DIS_ORIGIN AVG_TMP_DEG_ORIGIN AVG_DEW_DEG_ORIGIN AVG_SLP_ORIGIN AVG_WND_SPEED_DEST AVG_CIG_HEIGHT_DEST MIN_CIG_HEIGHT_DEST AVG_VIS_DIS_DEST MIN_VIS_DIS_DEST AVG_TMP_DEG_DEST AVG_DEW_DEG_DEST AVG_SLP_DEST MINUTES_AFTER_MIDNIGHT_ORIGIN MINUTES_AFTER_MIDNIGHT_DEST incoming_delays avg_incoming_delay ABE 2015-01-01T11:00:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N846AS 5103 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 11433 1143302 31295 DTW Detroit, MI MI 26 Michigan 43 600 552 -8.0 0.0 0.0 -1 0600-0659 12.0 604 728 5.0 753 733 -20.0 0.0 0.0 -2 0700-0759 0.0 0.0 113.0 101.0 84.0 1.0 425.0 2 0 America/New_York America/Detroit 0 6 53 7 2015-01-01T06:00:00.000+0000 2015-01-01T07:53:00.000+0000 2015-01-01T12:53:00.000+0000 5 340 72517014737 LEHIGH VALLEY INTERNATIONAL A 72537094847 DETROIT METRO WAYNE COUNTY AI 9.123668620151711E-4 2014-12-30T09:00:00.000+0000 2015-01-01T09:00:00.000+0000 2015-01-01T08:51:00.000+0000 2015-01-01T09:00:00.000+0000 0.0 4.296207965657047 0 29.305555555555557 20813.470588235294 1524.0 16093.0 16093.0 -40.04255319148936 -118.08571428571429 10213.7 53.51190476190476 14911.850683405892 640.0 16090.811764705883 16000.0 -69.98837209302326 -122.32558139534883 10192.760952380953 660 773 0.0 0.0 ABE 2015-01-01T11:35:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N136EV 5579 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 635 628 -7.0 0.0 0.0 -1 0600-0659 13.0 641 831 3.0 854 834 -20.0 0.0 0.0 -2 0800-0859 0.0 0.0 139.0 126.0 110.0 1.0 692.0 3 0 America/New_York America/New_York 35 6 54 8 2015-01-01T06:35:00.000+0000 2015-01-01T08:54:00.000+0000 2015-01-01T13:54:00.000+0000 5 1086 72517014737 LEHIGH VALLEY INTERNATIONAL A 72219013874 HARTSFIELD-JACKSON ATLANTA IN 9.123668620151711E-4 2014-12-30T09:35:00.000+0000 2015-01-01T09:35:00.000+0000 2015-01-01T08:51:00.000+0000 2015-01-01T08:52:00.000+0000 0.0 7.107549557148872 0 29.305555555555557 20813.470588235294 1524.0 16093.0 16093.0 -40.04255319148936 -118.08571428571429 10213.7 11.98913043478261 21082.112359550563 792.0 13132.08988764045 402.0 -10.024 -37.21348314606742 10275.547619047618 695 834 0.0 0.0 ABE 2015-01-01T21:00:00.000+0000 2015 1 1 1 4 2015-01-01 EV 20366 EV N131EV 5267 10135 1013503 30135 Allentown/Bethlehem/Easton, PA PA 42 Pennsylvania 23 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 1600 1600 0.0 0.0 0.0 0 1600-1659 9.0 1609 1752 7.0 1820 1759 -21.0 0.0 0.0 -2 1800-1859 0.0 0.0 140.0 119.0 103.0 1.0 692.0 3 0 America/New_York America/New_York 0 16 20 18 2015-01-01T16:00:00.000+0000 2015-01-01T18:20:00.000+0000 2015-01-01T23:20:00.000+0000 5 1086 72517014737 LEHIGH VALLEY INTERNATIONAL A 72219013874 HARTSFIELD-JACKSON ATLANTA IN 9.123668620151711E-4 2014-12-30T19:00:00.000+0000 2015-01-01T19:00:00.000+00

That worked. Let's do the whole thing.

In [0]:
# train

In [0]:
# verify row count doesn't change
data.count()

Out[24]: 16659291

In [0]:
data_with_zero_day = data.orderBy('ORIGIN_UTC').withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))
data_with_zero_day.registerTempTable('data')
count_incoming_delays = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, CONCAT(d1.ORIGIN, d1.ORIGIN_UTC, d1.TAIL_NUM, d1.DEST, d1.OP_CARRIER_FL_NUM) AS ID, COALESCE(sum(d2.ARR_DEL15),0) as CNT_INCOMING_DELAYS
      FROM data as d1
      LEFT JOIN data d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC, d1.TAIL_NUM, d1.DEST, d1.OP_CARRIER_FL_NUM
  """)

avg_incoming_delay = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, d1.TAIL_NUM, d1.DEST, d1.OP_CARRIER_FL_NUM, COALESCE(avg(d2.ARR_DELAY),0) as AVG_INCOMING_DELAY
      FROM sample_table as d1
      LEFT JOIN sample_table d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC, d1.TAIL_NUM, d1.DEST, d1.OP_CARRIER_FL_NUM
  """)

data_with_count = data.join(count_incoming_delays, ['ORIGIN', 'TAIL_NUM', 'ORIGIN_UTC', 'DEST', 'OP_CARRIER_FL_NUM'], 'left')
data_with_count_avg = data_with_count.join(avg_incoming_delay, ['ORIGIN', 'TAIL_NUM', 'ORIGIN_UTC', 'DEST', 'OP_CARRIER_FL_NUM'], 'left')

In [0]:
all_data = data
data = all_data.filter('YEAR == "2015"')

In [0]:
data_with_zero_day = data.orderBy('ORIGIN_UTC').withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))
data_with_zero_day.createOrReplaceTempView('data')
count_incoming_delays = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, CONCAT(d1.ORIGIN, d1.ORIGIN_UTC, d1.TAIL_NUM, d1.DEST, d1.OP_CARRIER_FL_NUM) AS ID, 
    COALESCE(sum(d2.ARR_DEL15),0) as CNT_INCOMING_DELAYS, COALESCE(avg(d2.ARR_DELAY),0) as AVG_INCOMING_DELAY
      FROM data as d1
      LEFT JOIN data d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC, ID
  """)

In [0]:
count_incoming_delays.count()

java.lang.RuntimeException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$3(DriverClient.scala:381)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:345)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:275)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:345)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:834)

In [0]:
count_incoming_delays.write.parquet('dbfs:/mnt/mids-w261/team20SSDK/airline/incoming_delays_train_2015')

In [0]:
# verify row count doesn't change
data_with_count_avg.count()

In [0]:
# one more sanity check before writing
display(data_with_count_avg)

In [0]:
# overwrite train
data_with_count_avg.write.mode('overwrite').parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/train_final")

In [0]:
# test

In [0]:
data = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/test_final")

In [0]:
# verify row count doesn't change
data.count()

In [0]:
data_with_zero_day = data.orderBy('ORIGIN_UTC').withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))
data_with_zero_day.createOrReplaceTempView('data')
count_incoming_delays = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(sum(d2.ARR_DEL15),0) as CNT_INCOMING_DELAYS
      FROM data as d1
      LEFT JOIN data d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

avg_incoming_delay = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(avg(d2.ARR_DELAY),0) as AVG_INCOMING_DELAY
      FROM sample_table as d1
      LEFT JOIN sample_table d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

data_with_count = data.join(count_incoming_delays, ['ORIGIN','ORIGIN_UTC'], 'left')
data_with_count_avg = data_with_count.join(avg_incoming_delay, ['ORIGIN','ORIGIN_UTC'], 'left')

In [0]:
# verify row count doesn't change
data_with_count_avg.count()

In [0]:
# one more sanity check before writing
display(data_with_count_avg)

In [0]:
# overwrite test
data_with_count_avg.write.mode('overwrite').parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/test_final")

In [0]:
# val

In [0]:
data = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/validation_final")

In [0]:
# verify row count doesn't change
data.count()

In [0]:
data_with_zero_day = data.orderBy('ORIGIN_UTC').withColumn("DAY_ZERO", f.date_trunc("day", "ORIGIN_UTC"))
data_with_zero_day.createOrReplaceTempView('data')
count_incoming_delays = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(sum(d2.ARR_DEL15),0) as CNT_INCOMING_DELAYS
      FROM data as d1
      LEFT JOIN data d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

avg_incoming_delay = spark.sql("""
    SELECT d1.ORIGIN, d1.ORIGIN_UTC, COALESCE(avg(d2.ARR_DELAY),0) as AVG_INCOMING_DELAY
      FROM sample_table as d1
      LEFT JOIN sample_table d2 
        ON d1.ORIGIN = d2.DEST 
          AND d2.DEST_UTC BETWEEN d1.DAY_ZERO AND d1.ORIGIN_UTC_ADJ_MAX
      GROUP BY d1.ORIGIN, d1.ORIGIN_UTC
  """)

data_with_count = data.join(count_incoming_delays, ['ORIGIN','ORIGIN_UTC'], 'left')
data_with_count_avg = data_with_count.join(avg_incoming_delay, ['ORIGIN','ORIGIN_UTC'], 'left')

In [0]:
# verify row count doesn't change
data_with_count_avg.count()

In [0]:
# one more sanity check before writing
display(data_with_count_avg)

In [0]:
# overwrite val
data_with_count_avg.write.mode('overwrite').parquet(f"dbfs:/mnt/mids-w261/team20SSDK/strategy/model_datasets/validation_final")